# CityBikes

Send a request to CityBikes for the city of your choice. 

In [ ]:
# Load Packages
```

import requests
import json
import re
import pandas as pd
import os # use this to access your environment variables
import plotly.express as px
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import sqlite3

# Test connection 
# Fetch the City Bikes API Data
request = requests.get('http://api.citybik.es/v2/networks')

# Check for success or failure
print(request.status_code)

# Observe the records
print(json.dumps(request.json(), sort_keys=True, indent=4))


# Ask the user to select a city of choice for analysis 

# Create a dictionary of href and city. City is required for user input and href is required for endpoint to fetch bike station data

dictionary_of_cities_href = {'list_of_cities': [],
                            'href': []}

for i in range(len(request.json()['networks'])):
    dictionary_of_cities_href['list_of_cities'].append(request.json()['networks'][i]['location']['city'])
    dictionary_of_cities_href['href'].append(request.json()['networks'][i]['href'])

# Print the list of cities for user to select 
user_choice = input(f"Select a city to fetch the data for:\n{', '.join(dictionary_of_cities_href['list_of_cities'])}\n")

pattern = re.compile(user_choice, re.IGNORECASE)

# Filter the list based on the regex match
matching_strings = [s for s in dictionary_of_cities_href['list_of_cities'] if pattern.search(s)]
    
print(f"You have selected - {matching_strings[0]}")

# Create and use the endpoint for the user input city to fetch the bike station data

endpoint = 'https://api.citybik.es/'+ dictionary_of_cities_href['href'][dictionary_of_cities_href['list_of_cities'].index(matching_strings[0])]

request_station_date = requests.get(endpoint)
```

print(request_station_date.status_code)

print(json.dumps(request_station_date.json(), sort_keys=True, indent=4))

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [ ]:
# Relevant Fields 
# Station_Id, Station_Name, Latitude, Longitude, Timestamp, City, Country, Total_Slots, Total Available_Bikes, Available Ebikes, 
# Available Normal Bikes

bike_dataframe_dictionary = {
    'Station_Id': [], # network (d) -> stations (ld) -> id (k)
    'Station_Name': [], # network (d) -> stations (ld) -> name (k)
    'Latitude': [], # network (d) -> stations (ld) -> latitude (k)
    'Longitude': [], # network (d) -> stations (ld) -> longitude (k) 
    'Timestamp': [], # network (d) -> stations (ld) -> timestamp (k)  
    'City': [], # network (d) -> location (d) -> city (k)
    'Country': [], # network (d) -> location (d) -> country (k)
    'Total_Available_Slots': [], # network (d) -> stations (ld) -> extra (d) -> slots (k)
    'Total_Available_Free_Bikes': [], # network (d) -> stations (ld) -> free_bikes (k)
    'Total_Available_EBikes': [], # network (d) -> stations (ld) -> extra (d) -> ebikes (k)
    'Total_Available_Normal_Bikes': [], # network (d) -> stations (ld) -> extra (d) -> normal_bikes (k)
    'Total_Available_Empty_Slots': [] # network (d) -> stations (ld) -> empty_slots (k)
}



for i in request_station_date.json()['network'].keys():
    if 'stations' in i:
        for j in range(len(request_station_date.json()['network'][i])):
            for k in request_station_date.json()['network'][i][j].keys():
                if 'id' == k:
                    bike_dataframe_dictionary['Station_Id'].append(request_station_date.json()['network'][i][j][k])
                if 'name' == k:
                    bike_dataframe_dictionary['Station_Name'].append(request_station_date.json()['network'][i][j][k])
                if 'latitude' == k:
                    bike_dataframe_dictionary['Latitude'].append(request_station_date.json()['network'][i][j][k])
                if 'longitude' == k:
                    bike_dataframe_dictionary['Longitude'].append(request_station_date.json()['network'][i][j][k])
                if 'timestamp' == k:
                    bike_dataframe_dictionary['Timestamp'].append(request_station_date.json()['network'][i][j][k])
                if 'empty_slots' == k:
                    bike_dataframe_dictionary['Total_Available_Empty_Slots'].append(request_station_date.json()['network'][i][j][k])
                if 'free_bikes' == k:
                    bike_dataframe_dictionary['Total_Available_Free_Bikes'].append(request_station_date.json()['network'][i][j][k])
                if 'extra' == k:
                    for m in request_station_date.json()['network'][i][j][k].keys():
                        if 'slots' == m:
                            bike_dataframe_dictionary['Total_Available_Slots'].append(request_station_date.json()['network'][i][j][k][m])
                        if 'ebikes' == m:
                            bike_dataframe_dictionary['Total_Available_EBikes'].append(request_station_date.json()['network'][i][j][k][m])
                        if 'normal_bikes' == m:
                            bike_dataframe_dictionary['Total_Available_Normal_Bikes'].append(request_station_date.json()['network'][i][j][k][m])
    if 'location' == i:
        for l in request_station_date.json()['network'][i].keys():
            if 'city' == l:
                bike_dataframe_dictionary['City'].append(request_station_date.json()['network'][i][l])
            if 'country' == l:
                bike_dataframe_dictionary['Country'].append(request_station_date.json()['network'][i][l])


Put your parsed results into a DataFrame.

In [ ]:
bike_dataframe_city_country = pd.DataFrame({'id': 1, 
                                           'City': bike_dataframe_dictionary['City'],
                                           'Country': bike_dataframe_dictionary['Country']})
bike_dataframe_station_details = pd.DataFrame({'id': 1, 
                                               'Station_Id': bike_dataframe_dictionary['Station_Id'],
                                                'Station_Name': bike_dataframe_dictionary['Station_Name'],
                                                'Latitude': bike_dataframe_dictionary['Latitude'],
                                                'Longitude': bike_dataframe_dictionary['Longitude'], 
                                                'Timestamp': bike_dataframe_dictionary['Timestamp'],  
                                                'Total_Available_Slots': bike_dataframe_dictionary['Total_Available_Slots'],
                                                'Total_Available_Free_Bikes': bike_dataframe_dictionary['Total_Available_Free_Bikes'],
                                                'Total_Available_EBikes': bike_dataframe_dictionary['Total_Available_EBikes'],
                                                'Total_Available_Normal_Bikes': bike_dataframe_dictionary['Total_Available_Normal_Bikes'],
                                                'Total_Available_Empty_Slots': bike_dataframe_dictionary['Total_Available_Empty_Slots']})
bike_dataframe = pd.merge(bike_dataframe_city_country, bike_dataframe_station_details, on = 'id', how = 'outer')

print(bike_dataframe.shape)
bike_dataframe 